In [12]:
import pandas as pd
from kiwipiepy import Kiwi
from kss import split_sentences
from collections import Counter
import pprint as pp
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

In [13]:
f = open("C:/Users/ghdtk/OneDrive/Desktop/VSCode/Python/KoreanNewsChronicle/Data/2019/2019 북미정상회담.txt", "r", encoding = 'utf-8')
content = f.read()

In [14]:
kiwi = Kiwi()
kiwi.load_user_dictionary('C:/Users/ghdtk/OneDrive/Desktop/VSCode/Python/KoreanNewsChronicle/PreProcess/new_wordlist2.txt')
kiwi.prepare()

stopword = []
def rm_stopword(stopword_file): 
    files = open(stopword_file, 'r', encoding="utf-8-sig")
    while True: #불용어 리스트 생성
        line = files.readline()
        if not line: break
        wordlist = line.split('\t')
        if wordlist[1].startswith('N'):
            stopword.append(wordlist[0])
    stopwords = set(stopword)
    files.close()


In [15]:
rm_stopword('C:/Users/ghdtk/OneDrive/Desktop/VSCode/Python/KoreanNewsChronicle/PreProcess/new_wordlist2.txt')
docs = []
merge_token = []
bi_gram = {}
LDA_Topics = []
i = 0
for news in open('C:/Users/ghdtk/OneDrive/Desktop/VSCode/Python/KoreanNewsChronicle/Data/2019/2019 북미정상회담.txt', encoding='utf-8'):
    doc = []
    context = []
    com = []
    lines = split_sentences(news)

    #형태소 분석
    for res in kiwi.analyze(lines):
        content = [word.strip() for word, tag, _, _ in res[0][0] if tag.startswith('N') and word not in stopword] #불용어사전 적용
#         context += content
        doc.append(content)
    docs.append(doc)

In [16]:
corpus = []
for Nouns in doc:
    Nouns = str(Nouns)
    Nouns = Nouns.replace("[", "").replace("]", "").replace("'", "")
    corpus.append(" ".join(Nouns.split(', ')))
c = " ".join(corpus)
corpus2 = []
corpus2.append(c)
corpus = corpus2

In [17]:
vectorizer = CountVectorizer()
sp_matrix = vectorizer.fit_transform(corpus)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

mylist = []
for i, sent in enumerate(corpus):
#     print('====== document[%d] ======' % i)
    mylist.append([(token, sp_matrix[i, word2id[token]]) for token in sent.split()])
#     print( [ (token, sp_matrix[i, word2id[token]]) for token in sent.split() ] )
    if i % 10000 == 0:
        print(i)

0


In [18]:
docs = []
for m in mylist:
    words = []
    values = []
    for noun in m:
        words.append(noun[0])
        values.append(noun[1])
    doc = pd.DataFrame({"word":words, "value":values})
    doc = doc.drop_duplicates(['word'])
    doc = doc.sort_values(by = 'value', ascending = False)
    docs.append(doc)

In [19]:
topics = []
for doc in docs:
    df2 = pd.DataFrame(doc)
    topic = []
    for i in range(len(df2)):
        topic.append(df2.iloc[i]['word'])
        if(i == 9):
            break
    topics.append(', '.join(topic))

In [20]:
oasis_topic = topics
oasis_topic

['회담, 영변, 결렬, 하노이, 대통령, 위원장, 조건, 정상, 외, 핵']

In [21]:
csv = pd.read_csv("C:/Users/ghdtk/OneDrive/Desktop/VSCode/Python/KoreanNewsChronicle/Data/2019/2019 북미정상회담-데이터10.csv")

In [22]:
text=csv['text']
print(text)

with open('News_Contents.txt','w',encoding='utf-8-sig') as file:
    for item in text:
        file.write("%s\n" % item.replace("\n"," "))


0    김정은 국무위원장은 2019년 신년사에서 “미국 대통령과 마주 앉을 준비가 되어 있...
1    27~28일 베트남 하노이에서 열리는 2차 북미정상회담을 앞두고 한국의 역할과 노력...
2    문재인 대통령은 2월 28일 18시50분부터 25분 동안 도널드 트럼프 미국 대통령...
3    한미연합훈련인 키리졸브(KR:Key Resolve) 연습과 독수리훈련(FE:Foal...
4    키리졸브연습은 한미 연합군사령부가 연합사 '작전계획 5027' 등을 적용해 컴퓨터 ...
5    일본의 주요 언론 매체들은 28일 베트남 하노이에서 열린 2차 북미 정상회담이 '하...
6    제2차 북미 정상회담이 합의에 이르지 못하고 결렬됐지만, 북한은 1일 이를 전혀 언...
7    도널드 트럼프 미국 대통령이 연두교서에서 제2차 북미 정상회담 날짜를 확정하면서 전...
8    김정은 북한 국무위원장이 베트남 하노이에서 열리는 2차 북미 정상회담에 참석하기 위...
9    김정은 북한 국무위원장에게 이번 2차 북미 정상회담은 그 결과에 따라 북한이 목표로...
Name: text, dtype: object


In [23]:
docs = []
merge_token = []
bi_gram = {}
LDA_Topics = []
i = 0
for news in open('News_Contents.txt', encoding='utf-8'):
    doc = []
    context = []
    com = []
    lines = split_sentences(news)

    #형태소 분석
    for res in kiwi.analyze(lines):
        content = [word.strip() for word, tag, _, _ in res[0][0] if tag.startswith('N') and word not in stopword] #불용어사전 적용
#         context += content
        doc.append(content)
    docs.append(doc)

In [24]:
corpus = []
for doc in docs:
    s = ""
    for Nouns in doc:
        Nouns = str(Nouns)
        Nouns = Nouns.replace("[", "").replace("]", "").replace("'", "")
        s = s + " ".join(Nouns.split(', '))
    corpus.append(s)

In [25]:
corpus

['김정 국무 위원장 년 신년사 대통령 준비 국제 사회 환영 결과 노력 것뒤 당 부위원장 방문 대통령 김 위원장 친서 전달차 북미 정상 회담 월 일 베트남 하노이 발표이 정상 년 월 일 싱가포르 역사 만남 지 일 만 일당 부위원장 대통령 한반도 평화 체제 구축 이후 주한 철수 요구 위원장 입장 공식 전달이 년 월 김용순 당 국제 부장 아널드 캔 터 차관 통일 뒤 지역 세력 균형 안보 주둔 필요 입장 전달 이래 측 전달 번 발언대북 제재 완화 협상 카드 그동안 북 간 협의 내용 국무 장관 대표 발언 수대표 월 일 스탠퍼드 대학 연설 전쟁 침공 것 정권 붕괴 시도 것 점 히월 일 의장 자리 개 이상 문제 논의 싱가포르 공동 성명 이행 협력 것 말국무 장관 월 일 인터뷰 대북 제재 완화 대가 결과 목표 대북 제재 완화 처음 협상 카드 제시월 일 인터뷰 전쟁 공식 종식 얘기 핵 한반도 안보 메커니즘 평화 메커니즘 창설 얘기이번 차 북미 정상 회담 차 회담 때 진전 핵 합의문 도출 것 기대협상 초기 일괄 타결 일괄 조치 접근법 부분 단계 접근 수용 선 핵 후 상응 조치 입장 견지반면 단계 동시 행동 조치 접근법 일관 주장월 일 대 북 정책 특별 대표 스탠퍼드 대학 연설 동시 병행 원칙 제시 측 접근법싱가포르 개 항 바탕 개 항 협의 월 일 평양 방문 특별 대표 대표 협의 차 북미 정상 회담 주 본격 협상 합의문 조율 것대표 싱가포르 공동 성명 개 항 기초 개 항목 협의 것 때 이번 하노이 공동 성명 핵 상응 조치 항목 포괄 합의 기대감 수논의 수준 때 이번 회담 한반도 핵 상응 조치 과정 이행 계획 포함 일괄 타결 빅딜 것핵 위협 본질 과거 핵 미래 핵 핵 포함 합의 스몰 디 국내외 반발 가능빅딜 실패 회담 절반 성공 회담 평가 것 스몰딜 것 성공 회담 평가 수차 북미 정상 회담 제약 현실 반영 한반도 완전 핵 비전 수 성공 회담 어디 합의 도출성공 회담 방식 최종 검증 비핵화 길목 확보언급 비핵화 핵심 과거 핵 처리 점불신 골 과거 핵 길목 합의 회담 성패 기준영변 

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict

vectorizer = CountVectorizer()
sp_matrix = vectorizer.fit_transform(corpus)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

mylist = []
for i, sent in enumerate(corpus):
#     print('====== document[%d] ======' % i)
    mylist.append([(token, sp_matrix[i, word2id[token]]) for token in sent.split()])
#     print( [ (token, sp_matrix[i, word2id[token]]) for token in sent.split() ] )
    if i % 10000 == 0:
        print(i)

0


In [27]:
docs = []
for m in mylist:
    words = []
    values = []
    for noun in m:
        words.append(noun[0])
        values.append(noun[1])
    doc = pd.DataFrame({"word":words, "value":values})
    doc = doc.drop_duplicates(['word'])
    doc = doc.sort_values(by = 'value', ascending = False)
    docs.append(doc)

In [28]:
topics = []
for doc in docs:
    df2 = pd.DataFrame(doc)
    topic = []
    for i in range(len(df2)):
        topic.append(df2.iloc[i]['word'])
        if(i == 9):
            break
    topics.append(', '.join(topic))

In [29]:
article_topics = topics

In [30]:
article_topics

['회담, 정상, 합의, 한반도, 조치, 북미, 포괄, 타결, 포함, 과거',
 '정상, 회담, 북미, 합의, 구축, 관련, 남북, 목표, 대표, 신뢰',
 '대통령, 협의, 회담, 정상, 위원장, 결과, 심도, 대화, 향후, 공조',
 '훈련, 장관, 결정, 통화, 이름, 연합, 한미, 것일, 때문올해, 종료',
 '훈련, 연합, 연습, 한미, 작전, 중단, 규모, 사령부, 명칭, 부대',
 '대통령, 회담, 기자, 회견, 속보, 합의, 뉴스, 제재, 내용, 해제',
 '회담, 위원장, 대통령, 사진, 정상, 대화, 결렬, 긍정, 진행, 단독',
 '회담, 정상, 위원장, 대통령, 북미, 조치, 합의, 진행, 핵무기, 협상',
 '베트남, 위원장, 오전, 시간, 열차, 전용, 국기, 선전, 인민, 출발',
 '경제, 위원장, 북미, 정상, 회담, 노선, 성장, 성과, 총력, 내부']

In [31]:
oasis_topic

['회담, 영변, 결렬, 하노이, 대통령, 위원장, 조건, 정상, 외, 핵']

In [34]:
from sklearn.metrics.pairwise import cosine_similarity
#코사인 유사도 threshold
cosine_similarity_threshold = [0.2, 0.4, 0.6, 0.8]

tfidf = CountVectorizer()

TP = [0, 0, 0, 0] # our's O oasis O
FP = [0, 0, 0, 0] # our's O oasis X
FN = [0, 0, 0, 0] # our's X oasis O

for topic in article_topics:
    topic_list = []
    topic_list.append(oasis_topic[0])
    topic_list.extend(article_topics)
    
    vec = tfidf.fit_transform(topic_list)
    
    similarity = cosine_similarity(vec[0], vec)[0][1:]
    for i in range(4):
        check = False
        for sim in similarity:
            if sim >= cosine_similarity_threshold[i]:
                check = True
                break
        if check:
            TP[i] += 1
        else:
            FP[i] += 1
            
for topic in oasis_topic[0]:
    topic_list = []
    topic_list.append(oasis_topic[0])
    topic_list.extend(article_topics)
    
    vec = tfidf.fit_transform(topic_list)
    
    similarity = cosine_similarity(vec[0], vec)[0][1:]
    for i in range(4):
        check = False
        for sim in similarity:
            if sim >= cosine_similarity_threshold[i]:
                check = True
                break
        if not check:
            FN[i] += 1
            
print(TP, FP, FN)

[10, 10, 0, 0] [0, 0, 10, 10] [0, 0, 39, 39]


In [35]:
print(similarity)

[0.2236068  0.2236068  0.4472136  0.         0.         0.2236068
 0.55901699 0.4472136  0.1118034  0.3354102 ]
